![GenAI Banner](https://raw.githubusercontent.com/ralf-42/Image/main/genai-banner-2.jpg)


<p><font size="5" color='grey'> <b>
Agenten
</b></font> </br></p>


---

In [ ]:
#@title 🔧 Umgebung einrichten{ display-mode: "form" }
!uv pip install --system -q git+https://github.com/ralf-42/GenAI.git#subdirectory=04_modul
from genai_lib.utilities import check_environment, get_ipinfo, setup_api_keys, mprint, install_packages
setup_api_keys(['OPENAI_API_KEY', 'HF_TOKEN', 'SERPAPI_API_KEY'], create_globals=False)
print()
check_environment()
print()
get_ipinfo()

In [ ]:
#@title 🛠️ Installationen { display-mode: "form" }
install_packages([('google-search-results', 'serpapi'), 'wikipedia'])

# 1 | Was ist ein echter Agent?
---

Wenn man sich mit generativer KI beschäftigt, stößt man früher oder später auf den Begriff **Agent** – also ein System, das Aufgaben eigenständig ausführt. Doch was genau ist ein *„echter“* Agent? Muss er vollständig autonom sein? Solche Fragen führen schnell zu endlosen Grundsatzdiskussionen – und genau das ist nicht hilfreich, wenn man einfach anfangen möchte, mit GenAI zu arbeiten.



<img src="https://raw.githubusercontent.com/ralf-42/Image/main/agentisch_1.png" class="logo" width="500"/>



Der bekannte KI-Experte **Andrew Ng** hat einen anderen, praktischeren Vorschlag gemacht: Statt darüber zu streiten, ob etwas ein Agent ist oder nicht, sollten wir lieber davon sprechen, wie **agentisch** ein System ist – also wie **selbstständig** es arbeitet. So kann man sich auf das konzentrieren, was wirklich zählt: Was kann das System leisten, und wo kann es sinnvoll eingesetzt werden?




<img src="https://raw.githubusercontent.com/ralf-42/Image/main/agentisch_2.png" class="logo" width="900"/>



Besonders einfach ist der Einstieg bei eher einfachen Aufgaben – also bei Prozessen, die heute noch manuell erledigt werden, wie das Ausfüllen von Formularen, das Nachschlagen in einer Datenbank oder das Kopieren von Informationen zwischen verschiedenen Systemen. Diese Aufgaben lassen sich gut in sogenannte agentische Workflows überführen – also in Abläufe, bei denen die KI (teilweise) selbstständig handelt.


Natürlich gibt es auch deutlich komplexere Anwendungen, bei denen die KI viele Entscheidungen trifft, Schleifen durchläuft und sich an neue Situationen anpasst. Solche Systeme sind spannend – aber gerade für den Anfang ist es oft sinnvoller, mit kleineren, überschaubaren Schritten zu starten. Dort liegen aktuell auch die meisten Chancen, GenAI im Alltag oder im Beruf sinnvoll einzusetzen.



**Fazit:**     
Es muss nicht gleich ein „superintelligenter“ Agent sein. Besser ist es, pragmatisch zu denken, einfache Prozesse zu automatisieren – und so Schritt für Schritt Erfahrungen zu sammeln.



<br>

**Agenten erweitern LLMs**

Agenten sind mehr als nur Sprachmodelle. Sie verbinden die Sprachfähigkeiten eines LLMs mit praktischer Handlungsfähigkeit:

+ LLM als Denkmodul: Das LLM übernimmt das Sprachverständnis und das logische Schlussfolgern.

+ Erweiterbarkeit durch Tools: Agenten greifen z. B. auf Web, Datenbanken oder externe APIs zu.

+ Transparenter Denkprozess: Der Agent zeigt, wie er zum Ergebnis kommt – ideal zum Lernen und Verstehen.

+ Entscheidungen treffen: Agenten analysieren, planen und wählen aus mehreren Optionen – nicht nur einmal, sondern iterativ.

# 2 | Direkter Vergleich
---

## 2.1 Setup und Tools

Bevor wir vergleichen können, müssen wir die notwendigen Tools für unseren Agenten definieren. Diese Tools repräsentieren die erweiterten Fähigkeiten, die einem einfachen LLM fehlen.



In [ ]:
# Einfache Tools definieren
from langchain_core.tools import Tool
from langchain_community.utilities.serpapi import SerpAPIWrapper
import requests

# ---- Tool 1 - calculator
def simple_calculator(expression):
    """Einfacher Rechner - Das kann ein LLM oft nicht präzise"""
    try:
        # Sicherheitscheck
        if any(x in expression for x in ['import', 'exec', '__']):
            return "Unsichere Operation"
        result = eval(expression)
        return f"{expression} = {result}"
    except:
        return "Berechnungsfehler"

# ---- Tool 2 - internet_search
serpapi = SerpAPIWrapper()

# ---- Tool 3 - weather_open_meteo
def get_weather_open_meteo(address):
    """Aktuelle Wetterdaten über Open-Meteo API"""
    try:
        geo_url = "https://nominatim.openstreetmap.org/search"
        params = {"q": address, "format": "json", "limit": 1}
        headers = {"User-Agent": "Wetterdaten-Skript/1.0 (kontakt@example.com)"} # Good practice
        geo_response = requests.get(geo_url, params=params, headers=headers).json()

        if geo_response:
            lat, lon = geo_response[0]["lat"], geo_response[0]["lon"]
            # Request Celsius temperature
            weather_url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true&temperature_unit=celsius&windspeed_unit=kmh"
            weather_response = requests.get(weather_url).json()

            if 'current_weather' in weather_response:
                current_weather = weather_response['current_weather']
                temp_c = current_weather.get('temperature', 'N/A')
                windspeed_kmh = current_weather.get('windspeed', 'N/A')
                weathercode = current_weather.get('weathercode', 'N/A') # You might want to map weathercode to description
                return f"Wetter in {address}: Temperatur {temp_c}°C, Windgeschwindigkeit {windspeed_kmh} km/h, Wettercode: {weathercode}"
            else:
                 return f"Keine Wetterdaten gefunden für {address}"
        else:
            return f"Fehler beim Abrufen der Geodaten für {address}"
    except Exception as e:
        return f"Fehler beim Abrufen der Wetterdaten: {e}"


# Tool-Liste erstellen
tools = [
    Tool(
        name="calculator",
        func=simple_calculator,
        description="🔢 RECHNER - Präzise Berechnungen (was LLM oft falsch macht)"
    ),
    Tool(
        name="internet_search",
        func=serpapi.run,
        description="🌐 INTERNETSUCHE - Aktuelle Informationen finden (was LLM NICHT kann)"
    ),
     Tool(
        name="weather_open_meteo",
        func=get_weather_open_meteo,
        description="🌤️ WETTER (Open-Meteo) - Echtzeitdaten abrufen (was LLM unmöglich ist). Input ist die Adresse oder der Ort."
    )
]

print("✅ Tools definiert:")
for tool in tools:
    print(f"   • {tool.name:25s}: {tool.description}")

## 2.2 Vergleichstest

Wir verwenden eine Frage, die sowohl aktuelle Daten als auch eine Berechnung erfordert, um die Grenzen eines LLMs und die Stärken eines Agenten zu demonstrieren.

<p><font color='black' size="5">
Basismodell
</font></p>

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent

import time

# LLM Setup
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0)

# Test-Frage die Grenzen aufzeigt
test_question = "Wie ist das Wetter in Berlin, was ist 2847 * 1923, wie ist der aktuelle XETRA Kurs der Aktie von Rheinmetall?"

mprint("### 🧪 VERGLEICHSTEST")
mprint("---")
mprint(f"**Frage:** {test_question}")
print()

<p><font color='black' size="5">
LLM only
</font></p>

In [ ]:
# 1. EINFACHES LLM PROBIEREN
mprint("### 1️⃣ EINFACHES LLM:")
mprint("---")

start_time = time.time()
llm_response = llm.invoke(test_question)
llm_time = time.time() - start_time

mprint(f"**Antwort:** {llm_response.content}")
mprint(f"**Zeit:** {llm_time:.2f}s")
print()
mprint("### ❌ PROBLEME:")
mprint("+ Keine aktuellen Wetterdaten")
mprint("+ Berechnung möglicherweise ungenau")
mprint("+ Kann nicht überprüfen ob Antwort stimmt")
print()

<p><font color='black' size="5">
Agent
</font></p>

In [ ]:
# 2. AGENT MIT TOOLS
mprint("### 2️⃣ AGENT MIT TOOLS:")
mprint("---")

# Agent mit create_agent erstellen (LangChain 1.0+ API)
# Gibt ein CompiledStateGraph zurück (basiert auf LangGraph!)
agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt="Du bist ein hilfreicher Agent. Nutze Tools für aktuelle Daten und Berechnungen. Formatiere das Ergebnis im Markdown-Format, Formeln: $ Formel $",
    # debug=True  # ⚠️ Kann in Colab zu OutStream-Fehler führen
)

start_time = time.time()
# Input-Format: {"messages": [{"role": "user", "content": "..."}]}
agent_response = agent.invoke({"messages": [{"role": "user", "content": test_question}]})
agent_time = time.time() - start_time

mprint(f"## 🤖 KI-Agent: ")
mprint("---")
# Agent-Antwort extrahieren (letztes Message in messages-Liste)
if isinstance(agent_response, dict) and 'messages' in agent_response:
    last_message = agent_response['messages'][-1]
    output = last_message.content if hasattr(last_message, 'content') else str(last_message)
else:
    output = str(agent_response)

mprint(f"{output}")
mprint(f"**Zeit:** {agent_time:.2f}s")
print()
mprint("### ✅ VORTEILE:")
mprint("+ Aktuelle Wetterdaten abgerufen")
mprint("+ Präzise Berechnung durchgeführt")
mprint("+ Transparenter Denkprozess sichtbar")
mprint("+ Schritte nachvollziehbar")

## 2.3 Unterschiede



Die Unterschiede zwischen LLM und Agent lassen sich in fünf Kernbereichen zusammenfassen, die den Paradigmenwechsel von statischer zu dynamischer KI verdeutlichen.

**Vergleich der Fähigkeiten:**

| Aspekt | Einfaches LLM | Agent |
|--------|---------------|-------|
| **Aktuelle Daten** | ❌ Nur Trainingsdaten | ✅ Über Tools |
| **Berechnungen** | ⚠️ Oft ungenau | ✅ Präzise Tools |
| **Externe APIs** | ❌ Unmöglich | ✅ Beliebig erweiterbar |
| **Transparenz** | 🔒 Verborgen | 👁️ Sichtbar |
| **Erweiterbarkeit** | ❌ Statisch | ✅ Modular |



**🎯 FAZIT:**
- **Agent = LLM + Tools + Reasoning**
- ➡️ Aus reaktiv wird proaktiv
- ➡️ Aus statisch wird dynamisch
- ➡️ Aus isoliert wird vernetzt


# 3 | Hands-On: Agent bauen
---

⚠️ Debug-Hinweis

<details>


**Problem mit `debug=True` in Jupyter/Colab:**

Der Parameter `debug=True` kann in Jupyter/Colab zu folgendem Fehler führen:
```
AttributeError: 'OutStream' object has no attribute 'watch_fd_thread'
```

**Ursache:** LangGraph's Debug-Output versucht, auf Output-Streams zuzugreifen, die in Jupyter nicht korrekt initialisiert sind.

**Lösungen:**
1. **Einfachste Lösung:** `debug=True` Parameter entfernen oder auf `False` setzen
2. **Alternative:** Verwenden Sie `stream_mode="values"` statt `debug=True`:
   ```python
   for chunk in agent.stream(input, stream_mode="values"):
       print(chunk)
   ```
3. **Production:** Verwenden Sie LangSmith für professionelles Debugging

</details>

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage
from langchain_core.tools import Tool
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.utilities.serpapi import SerpAPIWrapper
import os

In [ ]:
# Tools definieren

# --- Tool 1 - Datei lesen
def read_file(filename):
    try:
        with open(filename, 'r') as f:
            return f.read()
    except:
        return f"Datei {filename} nicht gefunden"

# --- Tool 2 - Datei schreiben
def write_file(content):
    filename, text = content.split("|", 1)
    with open(filename, 'w') as f:
        f.write(text)
    return f"Datei {filename} geschrieben"

# ---- Tool 3 - search
serpapi = SerpAPIWrapper()

# Tool 4 - Wikipedia-Suche
def wiki_search(term):
    wiki = WikipediaAPIWrapper()
    try:
        return wiki.run(term)
    except Exception as e:
        return f"Wikipedia-Fehler: {e}"

custom_tools = [
    Tool(name="read_file", func=read_file, description="Datei lesen"),
    Tool(name="write_file", func=write_file, description="Datei schreiben (Format: datei.txt|inhalt)"),
    Tool(name="search", func=serpapi.run, description="INTERNETSUCHE - Aktuelle Informationen finden (was LLM NICHT kann"),
    Tool(name="wiki", func=wiki_search, description="Frage einen Begriff in Wikipedia nach")
]

In [ ]:
# Prompt als Template definieren
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher Assistent mit Zugriff auf Tools."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# Modell definieren
model_name = "gpt-4o-mini"
temperature = 0.0

llm = ChatOpenAI(model=model_name, temperature=temperature)

In [ ]:
# 🔧 Agenten-Logik
# Agent mit create_agent erstellen (LangChain 1.0+ API)
custom_agent = create_agent(
    model=llm,
    tools=custom_tools,
    system_prompt="Du bist ein hilfreicher Assistent mit Zugriff auf Tools.",
    # debug=True kann Fehler in Colab/Jupyter verursachen
)

In [ ]:
# 🔧 Test
from datetime import date
today = date.today().strftime("%d.%m.%Y")

input_text = f"""
Erstelle eine Datei 'notiz.txt' mit dem Inhalt 'Agenten können autonom agieren. 🤖'.
Wie ist der aktuelle XETRA Kurs der Aktie von Rheinmetall vom {today}?
Lese die Datei 'notiz.txt' und verändere die Zeitform von Gegenwart in Zukunft.
Was ist die AKTUELLE Version von Python, die heute vom {today} installiert werden könnte?
Was steht zu Taylor Swift auf Wikipedia.
"""

# Agent aufrufen mit korrektem Input-Format
response = custom_agent.invoke({"messages": [{"role": "user", "content": input_text}]})

In [ ]:
mprint("## 🛠️ Hands-On Agent")
mprint("---")

# Extrahiere letzte Nachricht
if isinstance(response, dict) and 'messages' in response:
    last_message = response['messages'][-1]
    output = last_message.content if hasattr(last_message, 'content') else str(last_message)

    # Alle Messages anzeigen (um den Prozess zu sehen)
    mprint("### Agent-Verlauf:")
    for i, msg in enumerate(response['messages'], 1):
        role = getattr(msg, 'type', 'unknown')
        content_preview = str(msg.content if hasattr(msg, 'content') else msg)[:100]
        mprint(f"{i}. {role}: {content_preview}...")

    mprint("")
    mprint("### Finale Antwort:")
    mprint(output)
else:
    mprint("### Output:")
    mprint(str(response))

# 4 | Anatomie eines Agenten
---



Um Agenten effektiv einsetzen zu können, muss man  ihre innere Struktur verstehen. Ein Agent besteht aus vier Hauptkomponenten, die zusammenarbeiten, um komplexe Aufgaben zu lösen. Diese Architektur ermöglicht es, die Stärken von LLMs mit praktischen Werkzeugen zu kombinieren.

**Die 4 Kern-Komponenten**

Jede Komponente eines Agenten hat eine spezifische Rolle im Gesamtsystem. Das Verständnis dieser Rollen hilft beim Design eigener Agenten und bei der Fehlersuche.

+ **LLM (Das Gehirn)**: Das Large Language Model fungiert als zentrale Intelligenz des Agenten. Es versteht die Benutzeranfrage, interpretiert Tool-Ergebnisse und entscheidet über nächste Schritte. Ohne das LLM wäre der Agent nur eine Sammlung unverbundener Werkzeuge.

+ **Tools (Die Hände)**: Tools sind spezialisierte Funktionen, die dem Agenten erlauben, mit der Außenwelt zu interagieren. Sie können so einfach sein wie ein Rechner oder so komplex wie eine Datenbankverbindung. Jedes Tool erweitert die Fähigkeiten des Agenten erheblich.

+ **Agent-Scratchpad (Das Gedächtnis)**: Das Scratchpad speichert Zwischenergebnisse und den Verlauf der Tool-Aufrufe. Es ermöglicht dem Agenten, aus vorherigen Schritten zu lernen und komplexe, mehrstufige Reasoning-Prozesse durchzuführen.

+ **Executor (Der Koordinator)**: Der Executor orchestriert das Zusammenspiel aller Komponenten. Er entscheidet, wann Tools aufgerufen werden, wann genug Informationen gesammelt wurden und wann die finale Antwort generiert werden soll.


<br>

**Der Agent-*Denkprozess***

Der *Denkprozess* eines Agenten ist fundamental anders als der eines LLMs. Während ein LLM linear von Eingabe zu Ausgabe arbeitet, durchläuft ein Agent einen iterativen Zyklus aus Reasoning, Tool-Nutzung und Bewertung.

+ **Schritt 1 - Verstehen**: Der Agent analysiert die Benutzeranfrage und identifiziert, welche Informationen oder Aktionen benötigt werden. Dies geschieht durch das LLM, das die natürliche Sprache interpretiert.

+ **Schritt 2 - Planen**: Basierend auf dem Verständnis der Anfrage plant der Agent, welche Tools in welcher Reihenfolge verwendet werden sollten. Diese Planung kann sich während der Ausführung ändern.

+ **Schritt 3 - Ausführen**: Der Agent ruft das gewählte Tool auf und erhält ein Ergebnis. Dieses Ergebnis wird im Scratchpad gespeichert und steht für weitere Entscheidungen zur Verfügung.

+ **Schritt 4 - Bewerten**: Nach jedem Tool-Aufruf bewertet der Agent, ob genügend Informationen vorliegen oder weitere Schritte notwendig sind. Diese Bewertung bestimmt den weiteren Verlauf.

+ **Schritt 5 - Iterieren oder Antworten**: Je nach Bewertung kehrt der Agent zu Schritt 2 zurück oder generiert die finale Antwort. Diese Flexibilität ermöglicht die Lösung komplexer, unvorhersehbarer Probleme.



# 5 | Wann braucht man einen Agent?
---

Die Entscheidung zwischen einem einfachen LLM und einem Agenten hängt von den spezifischen Anforderungen Ihrer Anwendung ab. Eine klare Entscheidungsmatrix hilft dabei, die richtige Technologie für den jeweiligen Anwendungsfall zu wählen und Ressourcen effizient einzusetzen.



Die Wahl der richtigen Technologie beginnt mit der Analyse der Aufgabenanforderungen. Während LLMs für viele Textverarbeitungsaufgaben ausreichen, sind Agenten unverzichtbar, wenn externe Interaktionen oder aktuelle Daten benötigt werden.

**Verwenden Sie einen Agenten wenn:**

+ Sie **aktuelle** oder **dynamische** Daten benötigen, die sich häufig ändern (Aktienkurse, Wetter, Nachrichten). Agenten können über APIs auf Live-Daten zugreifen und diese in ihre Antworten integrieren.

+ Präzise Berechnungen erforderlich sind, bei denen **Genauigkeit** kritisch ist. LLMs approximieren mathematische Operationen, während Agenten echte Rechner-Tools verwenden.

+ **Externe** Systeme angesprochen werden müssen, wie Datenbanken, APIs oder andere Services. Agenten können diese Integrationen nahtlos abwickeln.

+ Komplexe, **mehrstufige Prozesse** durchgeführt werden sollen, bei denen jeder Schritt vom vorherigen abhängt. Der Agent-Reasoning-Loop ist für solche Szenarien optimiert.

**Ein einfaches LLM reicht wenn:**

+ Reine Textverarbeitung ohne externe Daten im Fokus steht. Für Zusammenfassungen, Übersetzungen oder Textanalysen sind LLMs optimal.

+ Kreative Aufgaben gelöst werden sollen, wie das Schreiben von Geschichten, Gedichten oder Marketing-Texten. Hier sind die kreativen Fähigkeiten des LLMs gefragt.

+ Erklärungen oder Bildungsinhalt basierend auf allgemeinem Wissen benötigt werden. LLMs haben Zugang zu einem enormen Wissensfundus.

+ Statische Code-Generierung ohne externe Abhängigkeiten erforderlich ist. Für einfache Programmieraufgaben sind LLMs sehr effektiv.

<br>

**Die Faustregel lautet**:    
Wenn Sie Tools, aktuelle Daten oder externe Interaktionen benötigen, wählen Sie einen Agenten. Für reine Textverarbeitung reicht ein LLM aus.

# A | Aufgabe
---

Die Aufgabestellungen unten bieten Anregungen, Sie können aber auch gerne eine andere Herausforderung angehen.

<p><font color='black' size="5">
Kalkulation
</font></p>

Gegeben ist eine Datei, die eine Reihe von Gleichungen enthält.
Der Dateiname ist GenAI/02 data/gleichungen.txt

**Gleichung:**    
41748459 - 87226336    
92995162 * 46769739    
61530438 * 56074589    
95329602 + 45418854    
412907 + 3731910    
...

Verwenden Sie einen LangChain-Agenten mit einem Tool, um jede dieser Gleichungen zu berechnen, und erstellen Sie eine Datei ähnlich dieser:

**Ergebnisse:**  
41748459 - 87226336 = 45477877   
92995162 * 46769739 = 4349359455002718   
61530438 * 56074589 = 3450294021839982   
95329602 + 45418854 = 140748456   
412907 + 3731910 = 4144817   
... ...

